In [22]:
import os
import json

parent = "/Users/finn/Documents/GitHub/FM-ds.github.io/E4E/work"

ensure_subtitle_space = True
update_descriptions = True

### Standardising Vega Parameters
The code in the cell below loops through all vega-lite specs in the parent path <b> and all sub-directories </b> and standardises the parameters listed in standardise_params. It preserve any parameters not specified in standardise_params that are not sub-parameters of a parameter specified in standardise_params (e.g. would preserve title in the example below). It <will not> preserve any sub-sub-parameters (e.g. would not preserve encoding.x.title if encoding.x.scale were specified in standard_params).

Additionally if ensure_subtitle_space, it ensure the final line of the subtitle is an empty line, adding one if needed. If update_descriptions is True, it will set the description of the spec to the subtitle, to ensure ARIA accesibility.

In [ ]:

standard_params = {
    "title" : {
        "fontSize": 16,
        "subtitleFontSize": 12,
        "subtitleFontStyle": "italic",
    }
}

# NB: This preserves subparameters that are not in the standard_params dict (e.g. would preserve "subtitle" and "title")
# But will not preserve sub-sub-parameters

# Recursively walk through the directory and subdirectories
for root, dirs, files in os.walk(parent):
    for file in [file for file in files if file.endswith(".json")]: # Only look at json files
        spec = json.load(open(os.path.join(root, file), "r"))
        if '$schema' in spec and "vega.github.io/schema" in spec['$schema']: # Only look at vega files
            for param in standard_params:
                if param not in spec:
                    spec[param] = standard_params[param]
                else:
                    for sub_param in standard_params[param]:
                        spec[param][sub_param] = standard_params[param][sub_param]
            # Also add a blank line to the subtitle if one is not present
            if "subtitle" in spec["title"]:
                if type(spec["title"]["subtitle"]) != list:
                    spec["title"]["subtitle"] = [spec["title"]["subtitle"], ""]
                elif len(spec["title"]["subtitle"][-1]) > 1:
                    spec["title"]["subtitle"].append("")
            json.dump(spec, open(os.path.join(root, file), "w")) # Overwrite the file
